<a href="https://colab.research.google.com/github/GustavoNicodemos/Algoritmo_analise_resultado/blob/main/Algoritmo_Anlise_GUV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

# ==============================================================
# Função para simular dados contábeis (apenas para testes iniciais)
# Substitua essa função pelo carregamento de seus dados reais
# ==============================================================
def simulate_accounting_data(n=500):
    np.random.seed(42)
    data = pd.DataFrame({
        'data': pd.date_range('2024-01-01', periods=n, freq='D'),
        'valor': np.random.normal(loc=1000, scale=300, size=n),
        'historico': np.random.choice(['Salário', 'Aluguel', 'Energia', 'Vendas', 'Serviços'], n),
        'centro_de_custo': np.random.choice(['ADM', 'VENDAS', 'TI'], n),
        'codigo_aglutinacao': np.random.choice(['DRE01', 'DRE02', 'DRE03'], n)
    })
    return data



In [ ]:
# ==============================================================
# Classe principal para análise contábil automatizada
# ==============================================================
class AccountingAnalyzer:
    def __init__(self):
        # Modelos utilizados na análise
        self.anomaly_detector = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
        self.clusterer = MiniBatchKMeans(n_clusters=5, random_state=42)
        self.encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
        self.classifier = xgb.XGBClassifier()
        self.history = pd.DataFrame()  # Armazena dados já analisados

    def analyze(self, new_data):
        """Executa a análise contábil nos dados fornecidos."""
        new_data = self._check_history(new_data)
        features = self._preprocess(new_data)

        # Detectar anomalias
        new_data['anomaly'] = self.anomaly_detector.fit_predict(features)

        # Agrupar em clusters
        new_data['cluster'] = self.clusterer.fit_predict(features)

        # Classificar tipo de problema se o modelo já foi treinado
        if not self.history.empty and hasattr(self.classifier, 'classes_'):
            model_features = self._create_features(new_data)
            new_data['problem_type'] = self.classifier.predict(model_features)

        # Atualiza o histórico
        self._update_history(new_data)

        # Retorna relatório consolidado
        return self._generate_report(new_data)

    def _check_history(self, data):
        """Remove dados já analisados anteriormente."""
        if self.history.empty:
            return data
        return data[~data.index.isin(self.history.index)]

    def _preprocess(self, data):
        """Codifica dados categóricos e prepara os dados numéricos."""
        categorical_data = self.encoder.fit_transform(data[['centro_de_custo', 'codigo_aglutinacao']])
        numerical_data = data[['valor']].values
        return np.hstack([numerical_data, categorical_data])

    def _create_features(self, data):
        """Gera features adicionais para o classificador."""
        categorical_data = self.encoder.transform(data[['centro_de_custo', 'codigo_aglutinacao']])
        numerical_data = data[['valor', 'anomaly', 'cluster']].values
        return np.hstack([numerical_data, categorical_data])

    def _update_history(self, data):
        """Adiciona os dados analisados ao histórico."""
        self.history = pd.concat([self.history, data], ignore_index=True)

    def _generate_explanations(self, data):
        """Gera justificativas automáticas para anomalias."""
        explanations = []
        for _, row in data.iterrows():
            if row['anomaly'] == -1:
                explanations.append(f"Valor atípico detectado em {row['centro_de_custo']} - {row['codigo_aglutinacao']}")
            else:
                explanations.append("Sem anomalias.")
        return explanations

    def _analyze_clusters(self, data):
        """Faz a análise dos agrupamentos por cluster."""
        return data.groupby(['cluster', 'codigo_aglutinacao']).agg({'valor': ['mean', 'count']}).reset_index()

    def _generate_report(self, data):
        """Compila o relatório final da análise."""
        return {
            'anomalies': data[data['anomaly'] == -1],
            'cluster_analysis': self._analyze_clusters(data),
            'explanations': self._generate_explanations(data)
        }



In [ ]:
# ==============================================================
# Execução de exemplo (substituir simulate_accounting_data por dados reais)
# ==============================================================
if __name__ == "__main__":
    dados_simulados = simulate_accounting_data()
    analisador = AccountingAnalyzer()
    relatorio = analisador.analyze(dados_simulados)

    # Armazena os dados em variáveis para visualização
    anomalias_df = relatorio['anomalies']
    clusters_df = relatorio['cluster_analysis']
    explicacoes_df = pd.DataFrame({'explicacao': relatorio['explanations']})

    # Exibe como DataFrames
    display(anomalias_df.head())
    display(clusters_df.head())
    display(explicacoes_df.head())